In [23]:
import pandas as pd
import numpy as np
dataset_file = r"dataset-3.csv"
#df = pd.read_csv(csv_file_path)

import pandas as pd

def calculate_distance_matrix(dataset_file):
    # Read the dataset into a DataFrame
    df = pd.read_csv(dataset_file)

    # Create a list of unique IDs
    unique_ids = sorted(set(df['id_start']).union(set(df['id_end'])))

    # Create an empty DataFrame for the distance matrix
    distance_matrix = pd.DataFrame(index=unique_ids, columns=unique_ids)

    # Initialize the distance matrix with 0
    distance_matrix = distance_matrix.fillna(0)

    # Fill in the known distances
    for index, row in df.iterrows():
        distance_matrix.loc[row['id_start'], row['id_end']] = row['distance']
        distance_matrix.loc[row['id_end'], row['id_start']] = row['distance']  # Accounting for bidirectional distances

    # Calculate the cumulative distances
    for i in unique_ids:
        for j in unique_ids:
            for k in unique_ids:
                if distance_matrix.loc[i, k] != 0 and distance_matrix.loc[k, j] != 0:
                    if distance_matrix.loc[i, j] == 0 or distance_matrix.loc[i, j] > distance_matrix.loc[i, k] + distance_matrix.loc[k, j]:
                        distance_matrix.loc[i, j] = distance_matrix.loc[i, k] + distance_matrix.loc[k, j]

    return distance_matrix

# Example usage
dataset_file = 'dataset-3.csv'
distance_matrix = calculate_distance_matrix(dataset_file)
print(distance_matrix)


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400     19.4      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7     19.4     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2     32.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0     32.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7     22.2     11.1     26.7   
1001410     78.7     69.0     48.8     32.8     11.1     22.2     15.6   
1001412     94.3     84.6     64.4     48.4     26.7     15.6     31.2   
1001414    112.5    102.8     82.6     66.6     44.9     33.8     18.2   
1001416    125.7    116.0     95.8     79.8     58.1     47.0     31.4   
1001418    139.3    129.6    109.4     93.4     71.7     60.6     45.0   
1001420    152.2    142.5    122.3    106.3     84.6     73.5     57.9   
1001422    161.8    152.1    131.9    115.9     94.2     83.1     67.5   
1001424    173.2    163.5    143.3    

In [50]:
# Question 2: Unroll Distance Matrix
def unroll_distance_matrix(distance_matrix):
    # Create an empty list to store the unrolled distance matrix
    unrolled_distance_data = []

    # Iterate through the distance matrix and extract id_start, id_end, and distance values
    for i in distance_matrix.index:
        for j in distance_matrix.columns:
            if i != j and distance_matrix.loc[i, j] != 0:
                unrolled_distance_data.append({'id_start': i, 'id_end': j, 'distance': distance_matrix.loc[i, j]})

    # Create a DataFrame from the unrolled distance data
    unrolled_distance_matrix = pd.DataFrame(unrolled_distance_data)

    return unrolled_distance_matrix
dataset_file = r"dataset-3.csv"
distance_matrix = calculate_distance_matrix(dataset_file)
unrolled_distance_matrix = unroll_distance_matrix(distance_matrix)
print(unrolled_distance_matrix)
 

     id_start   id_end  distance
0     1001400  1001402       9.7
1     1001400  1001404      29.9
2     1001400  1001406      45.9
3     1001400  1001408      67.6
4     1001400  1001410      78.7
..        ...      ...       ...
941   1004356  1001470     159.8
942   1004356  1001472     175.8
943   1004356  1001488       4.0
944   1004356  1004354       2.0
945   1004356  1004355       4.0

[946 rows x 3 columns]


In [51]:
# Question 3: Finding IDs within Percentage Threshold
def find_ids_within_ten_percentage_threshold(unrolled_distance_matrix, reference_value):
    # Calculate the average distance for the reference value
    average_distance = unrolled_distance_matrix[unrolled_distance_matrix['id_start'] == reference_value]['distance'].mean()

    # Calculate the threshold values
    lower_threshold = average_distance - (average_distance * 0.1)
    upper_threshold = average_distance + (average_distance * 0.1)

    # Find the IDs within the threshold
    ids_within_threshold = unrolled_distance_matrix[(unrolled_distance_matrix['distance'] >= lower_threshold) & (unrolled_distance_matrix['distance'] <= upper_threshold)]['id_start'].unique()

    # Sort the IDs
    ids_within_threshold = sorted(ids_within_threshold)

    return ids_within_threshold

# Example usage
reference_value = 1001400
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_distance_matrix, reference_value)
print(ids_within_threshold)

[1001400, 1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001420, 1001422, 1001424, 1001426, 1001428, 1001430, 1001432, 1001434, 1001436, 1001437, 1001438, 1001440, 1001442, 1001444, 1001446, 1001448, 1001470, 1001472, 1001488, 1004354, 1004355, 1004356]


In [52]:
#Question 4: Calculate Toll Rate
def calculate_toll_rate(unrolled_distance_matrix):
    # Create a copy of the input DataFrame
    toll_rate_matrix = unrolled_distance_matrix.copy()

    # Add columns for each vehicle type with their respective rate coefficients
    toll_rate_matrix['moto'] = 0.8
    toll_rate_matrix['car'] = 1.2
    toll_rate_matrix['rv'] = 1.5
    toll_rate_matrix['bus'] = 2.2
    toll_rate_matrix['truck'] = 3.6

    # Calculate toll rates for each vehicle type by multiplying the distance with the rate coefficient
    toll_rate_matrix['moto'] = toll_rate_matrix['distance'] * toll_rate_matrix['moto']
    toll_rate_matrix['car'] = toll_rate_matrix['distance'] * toll_rate_matrix['car']
    toll_rate_matrix['rv'] = toll_rate_matrix['distance'] * toll_rate_matrix['rv']
    toll_rate_matrix['bus'] = toll_rate_matrix['distance'] * toll_rate_matrix['bus']
    toll_rate_matrix['truck'] = toll_rate_matrix['distance'] * toll_rate_matrix['truck']

    return toll_rate_matrix

dataset_file = r"dataset-3.csv"
distance_matrix = calculate_distance_matrix(dataset_file)
unrolled_distance_matrix = unroll_distance_matrix(distance_matrix)
toll_rates = calculate_toll_rate(unrolled_distance_matrix)
toll_rates = toll_rates.drop(columns=['distance'])

# Print the output DataFrame
print(toll_rates)

     id_start   id_end    moto     car      rv     bus   truck
0     1001400  1001402    7.76   11.64   14.55   21.34   34.92
1     1001400  1001404   23.92   35.88   44.85   65.78  107.64
2     1001400  1001406   36.72   55.08   68.85  100.98  165.24
3     1001400  1001408   54.08   81.12  101.40  148.72  243.36
4     1001400  1001410   62.96   94.44  118.05  173.14  283.32
..        ...      ...     ...     ...     ...     ...     ...
941   1004356  1001470  127.84  191.76  239.70  351.56  575.28
942   1004356  1001472  140.64  210.96  263.70  386.76  632.88
943   1004356  1001488    3.20    4.80    6.00    8.80   14.40
944   1004356  1004354    1.60    2.40    3.00    4.40    7.20
945   1004356  1004355    3.20    4.80    6.00    8.80   14.40

[946 rows x 7 columns]
